In [3]:
import pandas as pd

# Результаты расчета в скрипте

Согласно README.md запустили следующий скрипт:

```bash
python3 -m venv dsp_env
source dsp_env/bin/activate
poetry install
pre-commit install
pre-commit run -a
python3 dsp_project/hw3_df2.py
```

Получили следующие метрики:

|    | file                      |   SNR, dB |      SDR |   SI-SDR |    PESQ |
|---:|:--------------------------|----------:|---------:|---------:|--------:|
|  0 | test_file_df2enhanced.wav |       nan | 36.5316  | 35.6604  | 4.52856 |
|  1 | mixed_-5_df2enhanced.wav  |        -5 |  4.37155 |  4.15107 | 1.43409 |
|  2 | mixed_0_df2enhanced.wav   |         0 |  9.04315 |  8.87922 | 1.86691 |
|  3 | mixed_5_df2enhanced.wav   |         5 | 12.8667  | 12.7457  | 2.50334 |
|  4 | mixed_10_df2enhanced.wav  |        10 | 16.35    | 16.2469  | 2.91191 |

Проведем расчет оставшихся метрик, а также заполним результаты MOS.

In [4]:
script_metric_vals = [
    {
        "PESQ": 4.528557777404785,
        "SI-SDR": 35.66038131713867,
        "SDR": 36.53158187866211,
        "file": "test_file_df2enhanced.wav",
        "SNR, dB": None,
    },
    {
        "PESQ": 1.4340887069702148,
        "SI-SDR": 4.151073455810547,
        "SDR": 4.371545791625977,
        "file": "mixed_-5_df2enhanced.wav",
        "SNR, dB": -5,
    },
    {
        "PESQ": 1.8669079542160034,
        "SI-SDR": 8.87922191619873,
        "SDR": 9.043149948120117,
        "file": "mixed_0_df2enhanced.wav",
        "SNR, dB": 0,
    },
    {
        "PESQ": 2.503340482711792,
        "SI-SDR": 12.745738983154297,
        "SDR": 12.866654396057129,
        "file": "mixed_5_df2enhanced.wav",
        "SNR, dB": 5,
    },
    {
        "PESQ": 2.9119064807891846,
        "SI-SDR": 16.2469482421875,
        "SDR": 16.349958419799805,
        "file": "mixed_10_df2enhanced.wav",
        "SNR, dB": 10,
    },
]

In [7]:
metric_vals_df = pd.DataFrame(script_metric_vals)[
    ["file", "SNR, dB", "SDR", "SI-SDR", "PESQ"]
]
metric_vals_df.to_csv("./hw3_metrics.csv")

# Расчет прочих метрик

## NISQA

1. Склонируем репозиторий:

```bash
git clone https://github.com/gabrielmittag/NISQA.git
```

2. В директории репозитория создадим и перейдем в environment из `env.yml`:

```bash
conda env create -f env.yml
conda activate nisqa
```

3. Положим файл `hw3_metrics.csv` в рабочую директорию, как и файлы со звуком. Запустим расчет метрик:

```bash
python run_predict.py --mode predict_csv --pretrained_model weights/nisqa.tar --csv_file hw3_metrics.csv --csv_deg file --num_workers 0 --bs 10 --output_dir .
```

In [9]:
nisqa_results = pd.read_csv("NISQA_results.csv").iloc[:, 2:-1]
nisqa_results.drop(["SDR", "SI-SDR", "PESQ"], axis=1, inplace=True)
nisqa_results

,"SNR, dB",mos_pred,noi_pred,dis_pred,col_pred,loud_pred
0,NaN,4.891995,4.652641,4.750807,4.525512,4.689187
1,-5.0,2.973273,3.746374,3.243140,3.000987,2.889263
2,0.0,4.114545,4.197488,4.294537,4.160984,4.112744
3,5.0,4.310910,4.212542,4.462294,4.247673,4.368694
4,10.0,4.448072,4.251392,4.510198,4.227535,4.404792


In [10]:
metric_vals_df = pd.merge(metric_vals_df, nisqa_results, on="SNR, dB")
metric_vals_df

,file,"SNR, dB",SDR,SI-SDR,PESQ,mos_pred,noi_pred,dis_pred,col_pred,loud_pred
0,test_file_df2enhanced.wav,NaN,36.531582,35.660381,4.528558,4.891995,4.652641,4.750807,4.525512,4.689187
1,mixed_-5_df2enhanced.wav,-5.0,4.371546,4.151073,1.434089,2.973273,3.746374,3.243140,3.000987,2.889263
2,mixed_0_df2enhanced.wav,0.0,9.043150,8.879222,1.866908,4.114545,4.197488,4.294537,4.160984,4.112744
3,mixed_5_df2enhanced.wav,5.0,12.866654,12.745739,2.503340,4.310910,4.212542,4.462294,4.247673,4.368694
4,mixed_10_df2enhanced.wav,10.0,16.349958,16.246948,2.911906,4.448072,4.251392,4.510198,4.227535,4.404792


## DNSMOS

1. Для расчета локально скопируем файл `dnsmos_local.py` и воспользуемся [инструкцией](https://github.com/microsoft/DNS-Challenge/tree/master/DNSMOS#to-use-the-local-evaluation-method). Для удобства можно предварительно перенести файлы в рабочую директорию:

```bash
python dnsmos_local.py -t . -o dnsmos_sample.csv
```

In [13]:
dnsmos_results = pd.read_csv("dnsmos_sample.csv").iloc[:, 1:]
dnsmos_results["filename"] = dnsmos_results["filename"].apply(
    lambda x: x[x.rfind("/") + 1 :]
)
dnsmos_results.columns = [
    "file",
    "len_in_sec",
    "sr",
    "num_hops",
    "OVRL_raw",
    "SIG_raw",
    "BAK_raw",
    "OVRL",
    "SIG",
    "BAK",
    "P808_MOS",
]
dnsmos_results

,file,len_in_sec,sr,num_hops,OVRL_raw,SIG_raw,BAK_raw,OVRL,SIG,BAK,P808_MOS
0,mixed_10_df2enhanced.wav,10.0,16000,1,3.934763,4.055880,4.555038,3.387534,3.575457,4.201802,4.048248
1,mixed_0_df2enhanced.wav,10.0,16000,1,3.292244,3.587282,3.993055,2.985019,3.304128,3.930007,3.785713
2,mixed_-5_df2enhanced.wav,10.0,16000,1,2.715870,3.067759,3.797256,2.576405,2.960206,3.815775,3.425663
3,test_file_df2enhanced.wav,10.0,16000,1,4.080358,4.229570,4.600471,3.470980,3.666659,4.220142,4.131903
4,mixed_5_df2enhanced.wav,10.0,16000,1,3.819027,3.984015,4.474706,3.319155,3.536239,4.168045,4.006088


In [14]:
metric_vals_df = pd.merge(
    metric_vals_df,
    dnsmos_results.drop(["sr", "len_in_sec", "num_hops"], axis=1),
    on="file",
)
metric_vals_df

,file,"SNR, dB",SDR,SI-SDR,PESQ,mos_pred,noi_pred,dis_pred,col_pred,loud_pred,OVRL_raw,SIG_raw,BAK_raw,OVRL,SIG,BAK,P808_MOS
0,test_file_df2enhanced.wav,NaN,36.531582,35.660381,4.528558,4.891995,4.652641,4.750807,4.525512,4.689187,4.080358,4.229570,4.600471,3.470980,3.666659,4.220142,4.131903
1,mixed_-5_df2enhanced.wav,-5.0,4.371546,4.151073,1.434089,2.973273,3.746374,3.243140,3.000987,2.889263,2.715870,3.067759,3.797256,2.576405,2.960206,3.815775,3.425663
2,mixed_0_df2enhanced.wav,0.0,9.043150,8.879222,1.866908,4.114545,4.197488,4.294537,4.160984,4.112744,3.292244,3.587282,3.993055,2.985019,3.304128,3.930007,3.785713
3,mixed_5_df2enhanced.wav,5.0,12.866654,12.745739,2.503340,4.310910,4.212542,4.462294,4.247673,4.368694,3.819027,3.984015,4.474706,3.319155,3.536239,4.168045,4.006088
4,mixed_10_df2enhanced.wav,10.0,16.349958,16.246948,2.911906,4.448072,4.251392,4.510198,4.227535,4.404792,3.934763,4.055880,4.555038,3.387534,3.575457,4.201802,4.048248


## MOS

- Численные результаты не фиксировали, но большинство опрашиваемых при обсуждении сошлось на том, что шум в `enhanced` вариантах стал тише, при этом сам голос будто потерял естественность: стал "булькающим", неровным.
- Также некоторые замечали, что для `mixed_-5_df2enhanced.wav` из-за тихой записи слишком сложно разобрать, что говорится. При этом для предыдущего варианта шум меньше мешал понять речь.
- Для вариантов с SNR > 0 db enhanced-записи имели больший успех при попарном сравнении.

**Итоговые получившиеся оценки:**

In [15]:
metric_vals_df

,file,"SNR, dB",SDR,SI-SDR,PESQ,mos_pred,noi_pred,dis_pred,col_pred,loud_pred,OVRL_raw,SIG_raw,BAK_raw,OVRL,SIG,BAK,P808_MOS
0,test_file_df2enhanced.wav,NaN,36.531582,35.660381,4.528558,4.891995,4.652641,4.750807,4.525512,4.689187,4.080358,4.229570,4.600471,3.470980,3.666659,4.220142,4.131903
1,mixed_-5_df2enhanced.wav,-5.0,4.371546,4.151073,1.434089,2.973273,3.746374,3.243140,3.000987,2.889263,2.715870,3.067759,3.797256,2.576405,2.960206,3.815775,3.425663
2,mixed_0_df2enhanced.wav,0.0,9.043150,8.879222,1.866908,4.114545,4.197488,4.294537,4.160984,4.112744,3.292244,3.587282,3.993055,2.985019,3.304128,3.930007,3.785713
3,mixed_5_df2enhanced.wav,5.0,12.866654,12.745739,2.503340,4.310910,4.212542,4.462294,4.247673,4.368694,3.819027,3.984015,4.474706,3.319155,3.536239,4.168045,4.006088
4,mixed_10_df2enhanced.wav,10.0,16.349958,16.246948,2.911906,4.448072,4.251392,4.510198,4.227535,4.404792,3.934763,4.055880,4.555038,3.387534,3.575457,4.201802,4.048248


In [16]:
metric_vals_df.to_csv("hw3_metrics.csv")